```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# cnvrg CORE with Minikube on WSL_Ubuntu

- https://minikube.sigs.k8s.io/docs/start/
- https://app.cnvrg.io/docs/guides/minikube.html#prerequisites-prepare-your-local-environment
- https://helm.sh/docs/intro/install/
- local deployment: https://app.cnvrg.io/docs/guides/helm-light-install.html
- this is not for local: https://app.cnvrg.io/docs/guides/helm-install.html

# Install Minikube
```
#curl -LO https://storage.googleapis.com/minikube/releases/v1.16.0/minikube-linux-amd64
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

# Test
minikube status
minikube start
kubectl get po -A

minikube dashboard

kubectl create deployment hello-minikube --image=k8s.gcr.io/echoserver:1.4
kubectl expose deployment hello-minikube --type=NodePort --port=8080

kubectl get services hello-minikube

# The easiest way to access this service is to let minikube launch a web browser for you
minikube service hello-minikube

# Your application is now available at http://localhost:7080/.
kubectl port-forward service/hello-minikube 7080:8080

minikube stop
minikube delete --all

```

## Setup Minikube Cluster
```
# Adjust available memory for Docker (not needed for WSL)

minikube config set EmbedCerts true

# Choose the right Kubernetes version 1.16.8 or earlier. 1.17 is not currently supported by cnvrg
minikube start --cpus=10 --memory='16g' --disk-size='30g' --kubernetes-version v1.16.8

minikube status

```

## Install Helm
https://helm.sh/docs/intro/install/
```
curl https://baltocdn.com/helm/signing.asc | sudo apt-key add -
sudo apt-get install apt-transport-https --yes
echo "deb https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
sudo apt-get update
sudo apt-get install helm
```

## Deploy cnvrg CORE using Helm
```
helm repo add cnvrg https://charts.cnvrg.io
helm repo update

# Get node name
kubectl get nodes

# Deploy
helm install cnvrg cnvrg/cnvrg -n cnvrg --create-namespace \
--timeout 1500s  --wait \
--set global.external_ip=$(minikube ip) \
--set global.node=minikube \
--set global.high_availability=false

# https://stackoverflow.com/questions/59443834/helm-3-install-for-resouces-that-exist
# List all helm releases
helm list --all --all-namespaces
# helm uninstall <release-name> -n <namespace>


kubectl get clusterrolebinding | grep cnvrg

usun
CLUSTER_IP=127.0.0.1
helm install cnvrg cnvrg/cnvrg -n cnvrg \
--creat-namespace \
--timeout 1500s \
--set networking.istio.externalIp=$(minikube ip) \
--set clusterDomain=$CLUSTER_IP.nip.io \
--set computeProfile=small


usun
helm install cnvrg cnvrg/cnvrg -n cnvrg --create-namespace --timeout 1500s \
--set networking.istio.externalIp=$(minikube ip).nip.io \
--set clusterDomain=$CLUSTER_IP \
--set global.external_ip \ 
--set computeProfile=small

# Monitor deployment
watch kubectl -n cnvrg get pods

# Get the ip address of the cluster
kubectl -n cnvrg get svc | grep ingress | awk "{print $4}"

# Not working (you can find IP and port in Docker Desktop in the Container)
# I get a different URL than stated in Completing the Setup (https://app.cnvrg.io/docs/guides/helm-light-install.html)
http://127.0.0.1:30080/

```

## Result (cannot access to app)
```
$ helm install cnvrg cnvrg/cnvrg -n cnvrg --create-namespace \
> --timeout 1500s  --wait \
> --set global.external_ip=$(minikube ip) \
> --set global.node=minikube \
> --set global.high_availability=false
NAME: cnvrg
LAST DEPLOYED: Thu Nov  4 21:26:57 2021
NAMESPACE: cnvrg
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
🚀 Thank you for installing cnvrg.io!

Your installation of cnvrg.io is now in progress.
Wait until all the pods in the cnvrg namespace will be ready (kubectl get pods -n cnvrg)
and access to the app via: http://app.
Join our community at https://community.cnvrg.io
Talk to our team via email at hi@cnvrg.io

---

$ helm list --all --all-namespaces
NAME    NAMESPACE       REVISION        UPDATED                                 STATUS          CHART           APP VERSION
cnvrg   cnvrg           1               2021-11-04 21:26:57.3734572 +0100 CET   deployed        cnvrg-2.25.0    2.25.0  

---

$ kubectl -n cnvrg get svc | grep ingress | awk "{print $4}"
istio-ingressgateway          LoadBalancer   10.107.254.46    <pending>     80:30351/TCP,443:32042/TCP              17m

---

$watch kubectl -n cnvrg get pods
Every 2.0s: kubectl -n cnvrg get pods                   ML: Thu Nov  4 21:34:37 2021
NAME                                           READY   STATUS      RESTARTS   AGE
app-799b99845b-l7kgc                           0/1     Init:0/4    0          6m27s
cnvrg-operator-6cf4964969-tf5xm                1/1     Running     0          7m40s
elastalert-c5fd97cf8-ghgnh                     1/1     Running     4          6m38s
elasticsearch-0                                1/1     Running     0          6m39s
fluentd-7t8t9                                  1/1     Running     0          6m37s
grafana-84cdf796dc-r6bp5                       1/1     Running     0          6m29s
hyper-7fcc7f597d-hw2t5                         1/1     Running     0          6m27s
istio-ingressgateway-8594dc7496-dhkcb          1/1     Running     0          6m45s
istio-operator-5bcc48b967-l44hm                1/1     Running     0          7m18s
istiod-8dbd659fb-7wdvn                         1/1     Running     0          6m57s
kibana-58f5fd99c6-xdqds                        1/1     Running     0          6m38s
kube-state-metrics-77664cb658-8p69r            3/3     Running     0          6m29s
minio-79b78959f4-cz64g                         1/1     Running     0          6m42s
minio-exporter-token-skgxn                     0/1     Completed   0          6m29s
mpi-operator-5b9c6b69fd-n9lmx                  1/1     Running     0          6m24s
node-exporter-tkgqk                            2/2     Running     0          6m30s
postgres-576f4d6667-xcln7                      1/1     Running     0          6m45s
prometheus-cnvrg-0                             3/3     Running     1          4m38s
prometheus-operator-57fb7dd7ff-hzkrg           2/2     Running     0          6m35s
redis-6fc69d7c46-qchv9                         1/1     Running     0          6m41s
searchkiq-55b79bd78b-m5plj                     0/1     Init:0/1    0          6m27s
sidekiq-fcc48477-4psx9                         0/1     Init:0/1    0          6m27s
sidekiq-fcc48477-nrwgl                         0/1     Init:0/1    0          6m27s
sidekiq-prometheus-exporter-7cb9dc8697-lvwjl   1/1     Running     0          6m29s
systemkiq-6648868845-d2d9z                     0/1     Init:0/1    0          6m27s
```

## Cleaning
```
# Upgrade cnvrg
helm upgrade cnvrg cnvrg/cnvrg

# Delete cnvrg
helm delete cnvrg 
kubectl delete namespace cnvrg

minikube stop

# Delete cluster
minikube delete
```